This notebook is a demonstration of PyBioPAX that contains a mix of functions for traversing Pathway Commons' full, detailed data dump.

Note: the notebook uses `pystow` and `tabulate` which are not dependencies of PyBioPAX and can both be separately installed from PyPI using `pip install`.

In [1]:
# Install packages if they're not already found
! python -c "import pystow" || pip install pystow
! python -c "import tabulate" || pip install tabulate

     ---------------------------------------- 97.9/97.9 KB 1.4 MB/s eta 0:00:00


Traceback (most recent call last):
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'pystow'
You should consider upgrading via the 'C:\Users\user\Documents\GitHub\PathSingle\venv\Scripts\python.exe -m pip install --upgrade pip' command.


Traceback (most recent call last):
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'tabulate'
You should consider upgrading via the 'C:\Users\user\Documents\GitHub\PathSingle\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
import gzip
import pickle
from collections import defaultdict, Counter
from typing import Optional, Iterable, Set, Tuple, Any

import pystow
from lxml import etree
from tabulate import tabulate
from tqdm.auto import tqdm
from IPython.display import HTML

import pybiopax
from pybiopax.biopax import *

In [3]:
def ensure_pc_detailed(version: Optional[str], force: bool = False):
    if version is None:
        import bioversions
        
        version = bioversions.get_version("pathwaycommons")
        print(f'BioPax version: {version}')

    url = f"https://www.pathwaycommons.org/archives/PC2/v{version}/PathwayCommons{version}.Detailed.BIOPAX.owl.gz"
    path = pystow.ensure("bio", "pathwaycommons", version, url=url)    
    return pybiopax.model_from_owl_gz(path)

pc12 = ensure_pc_detailed(version="12")

BadGzipFile: Not a gzipped file (b'<!')

A simple first look into a model is to count how many of each type of BioPAX element it contains.

In [3]:
type_counter = Counter(
    obj.__class__.__name__
    for obj in pc12.objects.values()
)

print(f"Got {sum(type_counter.values()):,} objects")

HTML(tabulate(type_counter.most_common(), tablefmt="html", headers=["Type", "Count"]))

Got 3,745,386 objects


Type,Count
TemplateReactionRegulation,962641
RelationshipXref,683543
Evidence,558049
TemplateReaction,166320
Control,152432
UnificationXref,133283
PublicationXref,119613
Protein,114023
SequenceSite,108257
Catalysis,100923


## Which proteins are in a phosphorylated state when catalyzing a reaction?

There are a few different perspectives for the concept of "active states", this is one way of identifying them. The first 15 are shown for brevity. To implement this check, we also implement the `iter_modifications()` function which gives all features for a given physical entity.

In [4]:
def iter_modifications(entity: PhysicalEntity, query: str) -> Iterable[ModificationFeature]:
    """Iterate over modification features in a protein that have the query string as a substring."""
    for feature in entity.feature or []:
        # If this is a modification feature which has a known type
        # and that type includes "phospho", i.e., is a phosphorylation
        if (
            isinstance(feature, ModificationFeature)
            and feature.modification_type
            and any(query in mod for mod in feature.modification_type.term)
        ):
            yield feature

def iter_phosphosites(protein: Protein):
    yield from iter_modifications(protein, "phospho")

rows = []
for obj in tqdm(pc12.get_objects_by_type(Catalysis)):
    for protein in obj.controller:
        if not isinstance(protein, Protein):
            continue
        features = list(iter_phosphosites(protein))
        if not features:
            continue
        rows.append((
            obj.display_name or "",
            protein.display_name,
            ", ".join(o.display_name for o in obj.controlled.left),
            ", ".join(o.display_name for o in obj.controlled.right),
        ))

print(f"Matched {len(rows)} examples")
        
HTML(tabulate(rows[:15], tablefmt="html", headers=["Name", "Enzyme", "Reactants", "Products"]))

0it [00:00, ?it/s]

Matched 1283 examples


Name,Enzyme,Reactants,Products
,RAF1,MEK1,MEK1
,JAK2_p,"STAT5, STAT5",STAT5_p
,RAF1,MEK2,MEK2
,JAK1_p,"gp130, gp130",gp130_p
,Akt_p,GSK3b,GSK3b_p
CATALYSIS,Raf1,MEK,MEK
,MEK1/2(MKK1/2)_p,ERK1/2,ERK1/2_p
,PLCgamma1,PI-4-5-P2,"DAG, IP3"
,MEK1/2(MKK1/2)_p,ERK1/2,ERK1/2_p
CATALYSIS,MAPKAPK2,SRF,SRF


## Which catalysys of biochemical reactions require a cofactor?

We find that Pathway Commons has a few more than 80 such annotations.

In [5]:
def iter_cofactored_catalyses(model: BioPaxModel) -> Iterable[Catalysis]:
    """Iterate over catalyses of biochemical reactions that require a cofactor."""
    for obj in model.get_objects_by_type(Catalysis):
        if not obj.cofactor:
            continue
        if not isinstance(obj.controlled, BiochemicalReaction):
            continue
        yield obj

rows = [
    (
        obj.display_name,
        obj.controller,
        obj.cofactor.display_name,
        ", ".join(o.display_name for o in obj.controlled.left),
        ", ".join(o.display_name for o in obj.controlled.right),
    )
    for obj in iter_cofactored_catalyses(pc12)
]

print(f"Matched {len(rows)} examples")

HTML(tabulate(rows[:15], tablefmt="html", headers=["Name", "Controller", "Cofactor", "Reactants", "Products"]))

Matched 84 examples


Name,Controller,Cofactor,Reactants,Products
tyrosine aminotransferase,[Complex(TAT)],pyridoxal-P,"2-oxoglutarate, tyr","glt, 4-hydroxyphenylpyruvate"
nitric oxide synthase,[Complex(iNOS)],protoheme IX,"NADPH, O2, arg, H+","L-citrulline, nitric oxide, NADP+, H2O"
protoporphyrinogen oxidase,[Complex(PPO)],FAD,"protoporphyrinogen IX, O2","hydrogen peroxide, protoporphyrin IX"
creatine kinase,[Complex(BB)],Mg2+,"creatine, ATP","creatine-phosphate, ADP, H+"
kynurenine 3-monooxygenase,[Protein(Kynurenine 3-monooxygenase)],FAD,"NADPH, O2, H+, L-kynurenine","3-hydroxy-L-kynurenine, NADP+, H2O"
arginase,[Complex(arginase type 2)],Mn2+,"arg, H2O","L-ornithine, urea"
methylarsonate reductase,"[Complex(glutathione transferase, &Omega; class)]",glutathione,"glutathione, methylarsonate","glutathione disulfide, methylarsonite"
glutamine-phenylpyruvate transaminase,[Complex(KAT1)],pyridoxal-P,"gln, 2-oxo-3-phenylpropanoate","phe, 2-oxoglutaramate"
<small>D</small>-lactate dehydrogenase (cytochrome),[Protein(DLD)],FAD,"an oxidized cytochrome c, (R)-lactate","a reduced c-type cytochrome, pyruvate, H+"
"indoleamine 2,3-dioxygenase",[Protein(IDO)],protoheme IX,"trp, O2, H+",N-formylkynurenine


## Find simple Phosphorylation Reactions

We now implement an iterator over the set of simple phoshporylation reactions in which there is a single protein reactant and product.

In [6]:
def is_simple_protein_reaction(obj: Any) -> bool:
    """Check if the object is a biochemical reaction with the same
    single protein entity as reactant/product.
    """
    if not isinstance(obj, BiochemicalReaction):
        return False
    if len(obj.left) != 1 or len(obj.right) != 1:
        return False
    left, right = obj.left[0], obj.right[0]
    if not isinstance(left, Protein) or not isinstance(right, Protein):
        return False
    if left.entity_reference != right.entity_reference:
        return False
    return True

def iter_simple_protein_reactions(model: BioPaxModel) -> Iterable[BiochemicalReaction]:
    """Iterate over biochemical reactions in the model that have the same
    single protein entity as reactant/product.
    """
    for obj in model.get_objects_by_type(BiochemicalReaction):
        if is_simple_protein_reaction(obj):
            yield obj

def iter_phosphorylations(m):
    for obj in iter_simple_protein_reactions(m):
        left = list(iter_phosphosites(obj.left[0]))
        right = list(iter_phosphosites(obj.right[0]))
        if not left and right:
            yield obj

rows = [
    (
        obj.display_name, obj.left[0]
    )
    for obj in iter_phosphorylations(pc12)
]

print(f"Matched {len(rows)} examples")

HTML(tabulate(rows[:15], tablefmt="html", headers=["Name", "Reactant"]))

Matched 15418 examples


Name,Reactant
STATE_TRANSITION_LEFT__Akt-RIGHT__Akt-_re22,Protein(Akt)
,Protein(PDHA2)
,Protein(AVEN)
,Protein(SIT1)
,Protein(RALY)
,Protein(ATP1A1)
,Protein(Kv11.1 iso5)
,Protein(HSF1)
,Protein(IRF3)
,Protein(TPPP)


## Get Proteins with Bound Small Molecules 

In general, complexes in the model can be obtained using `get_objects_by_type()`, and their components iterated over with the `component` attribute (note that for simplicity, here we do not resolve `member_entity_reference` attributes though generally these are important to traverse as well). The following functions iterate over complexes between proteins and one or more small molecules.

In [7]:
def head(it, n=10):
    for _, obj in zip(range(n), it):
        yield obj

def iter_bound(m):
    for obj in m.get_objects_by_type(Complex):
        c = Counter(c.__class__ for c in obj.component)
        if c.get(Protein) != 1:
            continue
        if {SmallMolecule, Protein} != set(c):
            continue
        yield obj

single_molecule = list(iter_bound(pc12))
        
print(f"Matched {len(single_molecule):,} examples\n")

for obj in head(single_molecule):
    print(obj)
    for component in obj.component:
        print(" ", component)

Matched 13,338 examples

Complex(phenanthrene/ESR2 protein complex)
  Protein(ESR2 protein)
  SmallMolecule(phenanthrene)
Complex(SIRT3(?-399):Zn2+)
  SmallMolecule(Zn2+)
  Protein(SIRT3)
Complex(mono-n-octyl phthalate/PPARB protein complex)
  SmallMolecule(mono-n-octyl phthalate)
  Protein(PPARB protein)
Complex(Cholestanols/GPBAR1 protein complex)
  SmallMolecule(Cholestanols)
  Protein(GPBAR1 protein)
Complex(RAB4A:GTP)
  Protein(RAB4A)
  SmallMolecule(GTP)
Complex(fenofibric acid/PPARA protein complex)
  SmallMolecule(fenofibric acid)
  Protein(PPARA protein)
Complex(Dipyridamole/RCAN1 protein alternative form complex)
  SmallMolecule(Dipyridamole)
  Protein(RCAN1 protein alternative form)
Complex(pirinixic acid/PPARA protein complex)
  SmallMolecule(pirinixic acid)
  Protein(PPARA protein)
Complex(4-nitrobenzylthioinosine/SLC29A1 protein complex)
  Protein(SLC29A1 protein)
  SmallMolecule(4-nitrobenzylthioinosine)
Complex(bisphenol A/PGR complex)
  Protein(PGR)
  SmallMolecule(bis

Get proteins with multiple bound small molecules

In [8]:
def iter_bound_multiple(m):
    for obj in m.get_objects_by_type(Complex):
        c = Counter(c.__class__ for c in obj.component)
        if c.get(Protein) != 1:
            continue
        if {SmallMolecule, Protein} != set(c):
            continue
        if c.get(SmallMolecule) < 2:
            continue
        yield obj

multiple_molecule = list(iter_bound_multiple(pc12))
        
print(f"Matched {len(multiple_molecule):,} examples\n")
        
for obj in head(multiple_molecule):
    print(obj)
    for component in obj.component:
        print(" ", component)

Matched 184 examples

Complex(Vitamin K 3/NADP/DCXR protein complex)
  SmallMolecule(Vitamin K 3)
  SmallMolecule(NADP)
  Protein(DCXR protein)
Complex(palmitoylated, myristoylated eNOS dimer)
  SmallMolecule(Zn2+)
  SmallMolecule(FMN)
  SmallMolecule(heme)
  Protein(2xPalmC-MyrG-NOS3)
  SmallMolecule(FAD)
Complex(Cholestanol/NAD/HSD17B10 protein complex)
  SmallMolecule(NAD)
  Protein(HSD17B10 protein)
  SmallMolecule(Cholestanol)
Complex(lauroyl-coenzyme A/NADP/HSDL2 protein complex)
  Protein(HSDL2 protein)
  SmallMolecule(lauroyl-coenzyme A)
  SmallMolecule(NADP)
Complex(CIAPIN1:4Fe-4S:2Fe-2S oxidized)
  Protein(CIAPIN1)
  SmallMolecule((2Fe-2S)(2+))
  SmallMolecule(4Fe-4S cluster)
Complex(pyruvate kinase complex, liver and RBC)
  Protein(Pyruvate kinase, R/L type)
  SmallMolecule(Mg2+)
  SmallMolecule(K+)
Complex(PP2B catalytic (Fe3+, Zn2+))
  SmallMolecule(Zn2+)
  SmallMolecule(Fe3+)
  Protein(PPP3CA,B,C)
Complex(Glycyrrhetinic Acid/NADP/HSD3B1 protein complex)
  SmallMolecule(Gl

## Protein Feature Counts

Find the protein with the most features

In [10]:
protein_feature_counter = Counter({
    protein: len(protein.feature)
    for protein in pc12.get_objects_by_type(Protein)
})

HTML(tabulate(
    protein_feature_counter.most_common(15), 
    tablefmt="html", 
    headers=["Protein", "Features"],
))

Protein,Modifications
Protein(NOTCH1 L1574Q Extracellular Fragment),39
Protein(NOTCH1 L1600P Extracellular Fragment),39
Protein(NOTCH1 I1616N Extracellular Fragment),39
Protein(NOTCH1 F1592S Extracellular Fragment),39
Protein(NOTCH1 R1598P Extracellular Fragment),39
Protein(NOTCH1 L1596H Extracellular Fragment),39
Protein(NOTCH1 I1616T Extracellular Fragment),39
Protein(NOTCH1 V1576E Extracellular Fragment),39
Protein(NOTCH1 L1593P Extracellular Fragment),39
Protein(NOTCH1 L1574P Extracellular Fragment),39


More specifically, the proteins with the most modification features

In [11]:
protein_mod_counter = Counter({
    protein: sum(isinstance(feature, ModificationFeature) for feature in protein.feature)
    for protein in pc12.get_objects_by_type(Protein)
})

HTML(tabulate(
    protein_mod_counter.most_common(15), 
    tablefmt="html", 
    headers=["Protein", "Modifications"],
))

Protein,Modifications
Protein(NOTCH1 L1574Q Extracellular Fragment),38
Protein(NOTCH1 L1600P Extracellular Fragment),38
Protein(NOTCH1 I1616N Extracellular Fragment),38
Protein(NOTCH1 F1592S Extracellular Fragment),38
Protein(NOTCH1 R1598P Extracellular Fragment),38
Protein(NOTCH1 L1596H Extracellular Fragment),38
Protein(NOTCH1 I1616T Extracellular Fragment),38
Protein(NOTCH1 V1576E Extracellular Fragment),38
Protein(NOTCH1 L1593P Extracellular Fragment),38
Protein(NOTCH1 L1574P Extracellular Fragment),38


## Interaction

Count the most interactions for each type of entity.

In [19]:
interaction_counter = Counter()
typed_interaction_counter = defaultdict(Counter)
for interaction in tqdm(pc12.get_objects_by_type(Interaction)):
    for participant in interaction.participant:
        interaction_counter[participant] += 1
        typed_interaction_counter[type(participant)][participant] += 1

HTML(tabulate(
    [
        (t.__name__, *row)
        for t, c in typed_interaction_counter.items()
        for row in c.most_common(1)
    ],
    tablefmt="html", 
    headers=["Entity", "Interactions"],
))

,Entity,Interactions
Rna,Rna(KTN1),947
Protein,Protein(AR),106
Complex,Complex(Histamine_Histamine H1 receptor (Homo sapiens)),80
SmallMolecule,SmallMolecule(Calcium),16
RnaRegion,RnaRegion(Isl-1),3
DnaRegion,DnaRegion(DAT),2
Dna,Dna(coding region),1
Pathway,<pybiopax.biopax.base.Pathway object at 0x23ba71460>,3
PhysicalEntity,PhysicalEntity(s66),1


## Parting Thoughts

We note that the high expressivity of BioPAX also sometimes enables the same
information to be encoded differently. For example, while the catalysis model
has an explicit slot for a cofactor, the same information could be encoded by
replacing the controller of the catalysis with a complex that includes the cofactor.
Ultimately, some interpretation may be required depending on the source of
BioPAX content. For example, the [INDRA BioPAX processor](https://indra.readthedocs.io/en/latest/modules/sources/biopax/index.html),
built on top of PyBioPAX, makes certain assumptions that trade some recall for
higher precision.
